In [0]:
from pyspark.sql.functions import col, lit, current_timestamp, sum as _sum
from delta.tables import DeltaTable
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite, VerificationResult
import os

print(os.environ['SPARK_VERSION'])
date_str = "2024-07-25"

booking_data = f"dbfs:/Volumes/incremental_load/default/orders_data/booking_data/bookings_{date_str}.csv"
customer_data = f"/Volumes/incremental_load/default/orders_data/customer_data/customers_{date_str}.csv"

print(booking_data)
print(customer_data)

booking_df = spark.read.format("csv").option("header", "True").option("inferSchema", "True").option("quote","\"").option("multiLine","True").load(booking_data)
booking_df.printSchema()
display(booking_df)

customer_df = spark.read.format("csv").option("header","True").option("inferSchema","True").option("quote","\"").option("multiLine","True").load(customer_data)
customer_df.printSchema()
display(customer_df)

